In [1]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

In [2]:
DB = {'servername': '.',
    'database': 'johari'}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
ssms_cursor = ssms_conn.cursor()
ssms_cursor

In [3]:
# Lokale connecties
sqlite_conn = sqlite3.connect('../data/processed/dwh.sqlite')
go_sales_conn = sqlite3.connect('../data/raw/go_sales.sqlite')
go_staff_conn = sqlite3.connect('../data/raw/go_staff.sqlite')
go_crm_conn = sqlite3.connect('../data/raw/go_crm.sqlite')

### SQLite --> Dataframes

In [4]:
# RETURNED_REASON
returned_reason = pd.read_sql_query('SELECT * FROM RETURN_REASON', go_sales_conn)
returned_reason = returned_reason[['RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']]
returned_reason.to_sql('RETURN_REASON', sqlite_conn, if_exists='replace', index=False)

5

In [5]:
# ORDER_METHOD
order_method = pd.read_sql_query('SELECT * FROM ORDER_METHOD', go_sales_conn)
order_method = order_method[['ORDER_METHOD_CODE', 'ORDER_METHOD_EN']]
order_method.to_sql('ORDER_METHOD', sqlite_conn, if_exists='replace', index=False)

7

In [6]:
# COURSE
course = pd.read_sql_query('SELECT * FROM COURSE', go_staff_conn)
course = course[['COURSE_CODE', 'COURSE_DESCRIPTION']]
course.to_sql('COURSE', sqlite_conn, if_exists='replace', index=False)

9

In [7]:
# SATISFACTION_TYPE
satisfaction_type = pd.read_sql_query('SELECT * FROM SATISFACTION_TYPE', go_staff_conn)
satisfaction_type = satisfaction_type[['SATISFACTION_TYPE_CODE', 'SATISFACTION_TYPE_DESCRIPTION']]
satisfaction_type.to_sql('SATISFACTION_TYPE', sqlite_conn, if_exists='replace', index=False)

5

In [8]:
# PRODUCT
product = pd.read_sql_query('SELECT * FROM PRODUCT', go_sales_conn)
product = product[[
    'PRODUCT_NUMBER', 
    'PRODUCT_NAME', 
    'PRODUCT_IMAGE', 
    'DESCRIPTION', 
    'PRODUCT_TYPE_CODE', 
    'INTRODUCTION_DATE', 
    'PRODUCTION_COST', 
    'MARGIN', 
    'LANGUAGE'
]]
product.to_sql('PRODUCT', sqlite_conn, if_exists='replace', index=False)

115

In [9]:
# SALES_STAFF
sales_staff = pd.read_sql_query('SELECT * FROM SALES_STAFF', go_staff_conn)
sales_branch = pd.read_sql_query('SELECT * FROM SALES_BRANCH', go_staff_conn)
country = pd.read_sql_query('SELECT * FROM COUNTRY', go_sales_conn)

sales_staff = sales_staff[['SALES_STAFF_CODE', 'FIRST_NAME', 'LAST_NAME', 'WORK_PHONE', 'EMAIL', 'FAX', 'SALES_BRANCH_CODE', 'POSITION_EN', 'DATE_HIRED', 'EXTENSION']]
sales_branch = sales_branch[['SALES_BRANCH_CODE', 'ADDRESS1', 'ADDRESS2', 'CITY', 'REGION', 'POSTAL_ZONE', 'COUNTRY_CODE']]
country = country[['COUNTRY_CODE', 'COUNTRY', 'LANGUAGE', 'CURRENCY_NAME']]

sales_staff = pd.merge(sales_staff, sales_branch, how='inner', left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')
sales_staff = pd.merge(sales_staff, country, how='inner', left_on='COUNTRY_CODE', right_on='COUNTRY_CODE')

sales_staff.to_sql('SALES_STAFF', sqlite_conn, if_exists='replace', index=False)

102

In [10]:
# RETAILER_CONTACT
retailer_contact = pd.read_sql_query('SELECT * FROM RETAILER_CONTACT', go_crm_conn)
retailer_site = pd.read_sql_query('SELECT * FROM RETAILER_SITE', go_crm_conn)
country_retailer_site = pd.read_sql_query('SELECT * FROM COUNTRY', go_crm_conn)
sales_territory_retailer_site = pd.read_sql_query('SELECT * FROM SALES_TERRITORY', go_crm_conn)
retailer = pd.read_sql_query('SELECT * FROM RETAILER', go_crm_conn)
retailer_type = pd.read_sql_query('SELECT * FROM RETAILER_TYPE', go_crm_conn)
retailer_headquarters = pd.read_sql_query('SELECT * FROM RETAILER_HEADQUARTERS', go_crm_conn)
country_retailer_headquarters = pd.read_sql_query('SELECT * FROM COUNTRY', go_crm_conn)
sales_territory_retailer_headquarters = pd.read_sql_query('SELECT * FROM SALES_TERRITORY', go_crm_conn)
retailer_segment = pd.read_sql_query('SELECT * FROM RETAILER_SEGMENT', go_crm_conn)
sales_demographic = pd.read_sql_query('SELECT * FROM SALES_DEMOGRAPHIC', go_crm_conn)
age_group = pd.read_sql_query('SELECT * FROM AGE_GROUP', go_crm_conn)


# Selecteer alleen de kolommen die we nodig hebben en hernoem ze wanneer nodig
retailer_contact = retailer_contact[['RETAILER_CONTACT_CODE', 'RETAILER_SITE_CODE', 'FIRST_NAME', 'LAST_NAME', 'JOB_POSITION_EN', 'EXTENSION', 'FAX', 'E_MAIL', 'GENDER']].rename(columns={'FAX': 'FAX_retailer_contact'})

retailer_site = retailer_site[['RETAILER_SITE_CODE', 'RETAILER_CODE', 'ADDRESS1', 'ADDRESS2', 'CITY', 'REGION', 'POSTAL_ZONE', 'COUNTRY_CODE', 'ACTIVE_INDICATOR']].rename(columns={'ADDRESS1': 'ADDRESS1_retailer_site', 'ADDRESS2': 'ADDRESS2_retailer_site', 'CITY': 'CITY_retailer_site', 'REGION': 'REGION_retailer_site', 'POSTAL_ZONE': 'POSTAL_ZONE_retailer_site'})
country_retailer_site = country_retailer_site[['COUNTRY_CODE', 'COUNTRY_EN', 'FLAG_IMAGE', 'SALES_TERRITORY_CODE']].rename(columns={'COUNTRY_EN': 'COUNTRY_EN_retailer_site', 'FLAG_IMAGE': 'FLAG_IMAGE_retailer_site', 'SALES_TERRITORY_CODE': 'SALES_TERRITORY_CODE_retailer_site'})
sales_territory_retailer_site = sales_territory_retailer_site[['SALES_TERRITORY_CODE', 'TERRITORY_NAME_EN']].rename(columns={'SALES_TERRITORY_CODE': 'SALES_TERRITORY_CODE_retailer_site', 'TERRITORY_NAME_EN': 'TERRITORY_NAME_EN_retailer_site'})

retailer = retailer[['RETAILER_CODE', 'RETAILER_CODEMR', 'COMPANY_NAME', 'RETAILER_TYPE_CODE']]
retailer_type = retailer_type[['RETAILER_TYPE_CODE', 'RETAILER_TYPE_EN']]

retailer_headquarters = retailer_headquarters[['RETAILER_CODEMR', 'RETAILER_NAME', 'ADDRESS1', 'ADDRESS2', 'CITY', 'REGION', 'POSTAL_ZONE', 'COUNTRY_CODE', 'PHONE', 'FAX', 'SEGMENT_CODE']].rename(columns={'FAX': 'FAX_retailer_headquarters', 'ADDRESS1': 'ADDRESS1_retailer_headquarters', 'ADDRESS2': 'ADDRESS2_retailer_headquarters', 'CITY': 'CITY_retailer_headquarters', 'REGION': 'REGION_retailer_headquarters', 'POSTAL_ZONE': 'POSTAL_ZONE_retailer_headquarters'})
country_retailer_headquarters = country_retailer_headquarters[['COUNTRY_CODE', 'COUNTRY_EN', 'FLAG_IMAGE', 'SALES_TERRITORY_CODE']].rename(columns={'COUNTRY_EN': 'COUNTRY_EN_retailer_headquarters', 'FLAG_IMAGE': 'FLAG_IMAGE_retailer_headquarters', 'SALES_TERRITORY_CODE': 'SALES_TERRITORY_CODE_retailer_headquarters'})
sales_territory_retailer_headquarters = sales_territory_retailer_headquarters[['SALES_TERRITORY_CODE', 'TERRITORY_NAME_EN']].rename(columns={'SALES_TERRITORY_CODE': 'SALES_TERRITORY_CODE_retailer_headquarters', 'TERRITORY_NAME_EN': 'TERRITORY_NAME_EN_retailer_headquarters'})
retailer_segment = retailer_segment[['SEGMENT_CODE', 'LANGUAGE', 'SEGMENT_NAME', 'SEGMENT_DESCRIPTION']]
sales_demographic = sales_demographic[['DEMOGRAPHIC_CODE', 'RETAILER_CODEMR', 'AGE_GROUP_CODE', 'SALES_PERCENT']]
age_group = age_group[['AGE_GROUP_CODE', 'UPPER_AGE', 'LOWER_AGE']]


# Merge alle tabellen
retailer_site = pd.merge(retailer_site, country_retailer_site, how='inner', on='COUNTRY_CODE').rename(columns={'COUNTRY_CODE': 'COUNTRY_CODE_retailer_site'})
retailer_site = pd.merge(retailer_site, sales_territory_retailer_site, how='inner', on='SALES_TERRITORY_CODE_retailer_site', suffixes=('', '_retailer_site'))
retailer_contact = pd.merge(retailer_contact, retailer_site, how='inner', on='RETAILER_SITE_CODE')
retailer_contact = pd.merge(retailer_contact, retailer, how='inner', on='RETAILER_CODE')
retailer_contact = pd.merge(retailer_contact, retailer_type, how='inner', on='RETAILER_TYPE_CODE')
retailer_contact = pd.merge(retailer_contact, retailer_headquarters, how='inner', on='RETAILER_CODEMR')
retailer_contact = pd.merge(retailer_contact, country_retailer_headquarters, how='inner', on='COUNTRY_CODE').rename(columns={'COUNTRY_CODE': 'COUNTRY_CODE_retailer_headquarters'})
retailer_contact = pd.merge(retailer_contact, sales_territory_retailer_headquarters, how='inner', on='SALES_TERRITORY_CODE_retailer_headquarters')
retailer_contact = pd.merge(retailer_contact, retailer_segment, how='inner', on='SEGMENT_CODE')
retailer_contact = pd.merge(retailer_contact, sales_demographic, how='inner', on='RETAILER_CODEMR')
retailer_contact = pd.merge(retailer_contact, age_group, how='inner', on='AGE_GROUP_CODE')


# Naar SQLite voor visuele debugging
retailer_contact.to_sql('RETAILER_CONTACT', sqlite_conn, if_exists='replace', index=False)

2082

### Dataframes --> SSMS

In [11]:
# RETURNED_REASON
for index, row in returned_reason.iterrows():
    try:
        query = f"INSERT INTO dbo.RETURN_REASON VALUES ({row['RETURN_REASON_CODE']}, '{row['RETURN_DESCRIPTION_EN']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        ssms_conn.rollback()
        break

In [12]:
# ORDER_METHOD
for index, row in order_method.iterrows():
    try:
        query = f"INSERT INTO dbo.ORDER_METHOD VALUES ({row['ORDER_METHOD_CODE']}, '{row['ORDER_METHOD_EN']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        ssms_conn.rollback()
        break

In [13]:
# COURSE
for index, row in course.iterrows():
    try:
        query = f"INSERT INTO dbo.COURSE VALUES ({row['COURSE_CODE']}, '{row['COURSE_DESCRIPTION']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        ssms_conn.rollback()
        break

In [14]:
# SATISFACTION_TYPE
for index, row in satisfaction_type.iterrows():
    try:
        query = f"INSERT INTO dbo.SATISFACTION_TYPE VALUES ({row['SATISFACTION_TYPE_CODE']}, '{row['SATISFACTION_TYPE_DESCRIPTION']}')"
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        ssms_conn.rollback()
        break

In [15]:
# PRODUCT
for index, row in product.iterrows():
    try:
        introduction_date = pd.to_datetime(row['INTRODUCTION_DATE'], dayfirst=True).strftime('%Y-%m-%d')
        query = f"""
        INSERT INTO dbo.PRODUCT 
        (product_product_id, product_name, product_image, product_line_name, product_type_name, product_introduction_date, product_cost, product_margin, product_language) 
        VALUES 
        ({row['PRODUCT_NUMBER']}, '{row['PRODUCT_NAME']}', '{row['PRODUCT_IMAGE']}', '', '{row['PRODUCT_TYPE_CODE']}', '{introduction_date}', '{row['PRODUCTION_COST']}', {row['MARGIN']}, '{row['LANGUAGE']}')
        """
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        ssms_conn.rollback()
        break

In [16]:
# SALES_STAFF
for index, row in sales_staff.iterrows():
    try:
        date_hired = pd.to_datetime(row['DATE_HIRED'], format='%d-%b-%Y %I:%M:%S %p').strftime('%Y-%m-%d')
        extension = 'NULL' if pd.isnull(row['EXTENSION']) else f"'{row['EXTENSION']}'"
        address1 = row['ADDRESS1'].replace("'", "''")  # Escape single quotes
        address2 = 'NULL' if pd.isnull(row['ADDRESS2']) else f"'{row['ADDRESS2'].replace("'", "''")}'"  # Escape single quotes
        city = row['CITY'].replace("'", "''")  # Escape single quotes
        region = 'NULL' if pd.isnull(row['REGION']) else f"'{row['REGION'].replace("'", "''")}'"  # Escape single quotes
        postal_zone = row['POSTAL_ZONE'].replace("'", "''")  # Escape single quotes
        country = row['COUNTRY'].replace("'", "''")  # Escape single quotes
        language = row['LANGUAGE'].replace("'", "''")  # Escape single quotes
        currency_name = row['CURRENCY_NAME'].replace("'", "''")  # Escape single quotes
        query = f"""
        INSERT INTO dbo.SALES_STAFF 
        (sales_staff_id, first_name, last_name, phone, email, fax, position_en, date_hired, extension, sales_branch_id, sales_branch_address1, sales_branch_address2, sales_branch_city, sales_branch_region, sales_branch_postal_zone, country_id, country_name, country_language, country_currency) 
        VALUES 
        ({row['SALES_STAFF_CODE']}, '{row['FIRST_NAME']}', '{row['LAST_NAME']}', '{row['WORK_PHONE']}', '{row['EMAIL']}', '{row['FAX']}', '{row['POSITION_EN']}', '{date_hired}', {extension}, '{row['SALES_BRANCH_CODE']}', '{address1}', {address2}, '{city}', {region}, '{postal_zone}', '{row['COUNTRY_CODE']}', '{country}', '{language}', '{currency_name}')
        """
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        ssms_conn.rollback()
        break

In [17]:
# RETAILER_CONTACT
for index, row in retailer_contact.iterrows():
    try:
        extension = row['EXTENSION'] if row['EXTENSION'] is not None else ''
        last_name = row['LAST_NAME'].replace("'", "''") if row['LAST_NAME'] is not None else ''
        city_retailer_site = row['CITY_retailer_site'].replace("'", "''") if row['CITY_retailer_site'] is not None else ''
        address1_retailer_site = row['ADDRESS1_retailer_site'].replace("'", "''") if row['ADDRESS1_retailer_site'] is not None else ''
        address2_retailer_site = row['ADDRESS2_retailer_site'].replace("'", "''") if row['ADDRESS2_retailer_site'] is not None else ''
        address1_retailer_headquarters = row['ADDRESS1_retailer_headquarters'].replace("'", "''") if row['ADDRESS1_retailer_headquarters'] is not None else ''
        address2_retailer_headquarters = row['ADDRESS2_retailer_headquarters'].replace("'", "''") if row['ADDRESS2_retailer_headquarters'] is not None else ''
        company_name = row['COMPANY_NAME'].replace("'", "''") if row['COMPANY_NAME'] is not None else ''
        retailer_name = row['RETAILER_NAME'].replace("'", "''") if row['RETAILER_NAME'] is not None else ''

        query = f"""
        INSERT INTO [johari].[dbo].[RETAILER_CONTACT]
        (
            [id],
            [first_name],
            [last_name],
            [job_position_en],
            [extension],
            [fax],
            [email],
            [gender],
            [retailer_site_id],
            [retailer_site_address1],
            [retailer_site_address2],
            [retailer_site_city],
            [retailer_site_region],
            [retailer_site_postal_zone],
            [retailer_site_country_id],
            [retailer_site_country_en],
            [retailer_site_flag_image],
            [retailer_site_sales_territory_id],
            [retailer_site_sales_territory_name_en],
            [retailer_id],
            [retailer_codemr],
            [retailer_company_name],
            [retailer_type_id],
            [retailer_type_type_en],
            [retailer_headquarters_id],
            [retailer_headquarters_name],
            [retailer_headquarters_address1],
            [retailer_headquarters_address2],
            [retailer_headquarters_city],
            [retailer_headquarters_region],
            [retailer_headquarters_postal_zone],
            [retailer_headquarters_phone],
            [retailer_headquarters_fax],
            [retailer_headquarters_segment_id],
            [retailer_segment_language],
            [retailer_segment_name],
            [retailer_segment_description],
            [retailer_headquarters_country_id],
            [retailer_headquarters_country_en],
            [retailer_headquarters_flag_image],
            [retailer_headquarters_sales_territory_id],
            [retailer_headquarters_sales_territory_name_en],
            [sales_demographic_id],
            [sales_demographic_codemr],
            [sales_demographic_sales_percent],
            [sales_demographic_age_group_id],
            [age_group_id],
            [age_group_upper_age],
            [age_group_lower_age]
        )
        VALUES
        (
            {row['RETAILER_CONTACT_CODE']},
            '{row['FIRST_NAME']}',
            '{last_name}',
            '{row['JOB_POSITION_EN']}',
            '{extension}',
            '{row['FAX_retailer_contact']}',
            '{row['E_MAIL']}',
            '{row['GENDER']}',
            '{row['RETAILER_SITE_CODE']}',
            '{address1_retailer_site}',
            '{address2_retailer_site}',
            '{city_retailer_site}',
            '{row['REGION_retailer_site']}',
            '{row['POSTAL_ZONE_retailer_site']}',
            '{row['COUNTRY_CODE_retailer_site']}',
            '{row['COUNTRY_EN_retailer_site']}',
            '{row['FLAG_IMAGE_retailer_site']}',
            '{row['SALES_TERRITORY_CODE_retailer_site']}',
            '{row['TERRITORY_NAME_EN_retailer_site']}',
            '{row['RETAILER_CODE']}',
            '{row['RETAILER_CODEMR']}',
            '{company_name}',
            '{row['RETAILER_TYPE_CODE']}',
            '{row['RETAILER_TYPE_EN']}',
            '{row['RETAILER_CODEMR']}',
            '{retailer_name}',
            '{address1_retailer_headquarters}',
            '{address2_retailer_headquarters}',
            '{row['CITY_retailer_headquarters']}',
            '{row['REGION_retailer_headquarters']}',
            '{row['POSTAL_ZONE_retailer_headquarters']}',
            '{row['PHONE']}',
            '{row['FAX_retailer_headquarters']}',
            '{row['SEGMENT_CODE']}',
            '{row['LANGUAGE']}',
            '{row['SEGMENT_NAME']}',
            '{row['SEGMENT_DESCRIPTION']}',
            '{row['COUNTRY_CODE_retailer_headquarters']}',
            '{row['COUNTRY_EN_retailer_headquarters']}',
            '{row['FLAG_IMAGE_retailer_headquarters']}',
            '{row['SALES_TERRITORY_CODE_retailer_headquarters']}',
            '{row['TERRITORY_NAME_EN_retailer_headquarters']}',
            '{row['DEMOGRAPHIC_CODE']}',
            '{row['RETAILER_CODEMR']}',
            '{row['SALES_PERCENT']}',
            '{row['AGE_GROUP_CODE']}',
            '{row['AGE_GROUP_CODE']}',
            '{row['UPPER_AGE']}',
            '{row['LOWER_AGE']}'
        )
        """
        ssms_cursor.execute(query)
        ssms_conn.commit()
    except Exception as e:
        print(e)
        print(row)
        ssms_conn.rollback()
        break